# Ensembling of Various Models: Voting and Stacking

#### Import Libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

#options for display
%matplotlib inline
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 120)

Ensemble Packages

In [6]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

##### Train and test data :)

In [14]:
train_x = pd.read_csv('../Data/train_x2.csv')
train_y = pd.read_csv('../Data/train_y2.csv',header=None)
train_y = train_y.values.ravel()
test_x = pd.read_csv('../Data/test_x2.csv')

###### Scoring Metric

In [15]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_x.values)
    rmse= np.sqrt(-cross_val_score(model, train_x.values, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

Libraries for specific Ensembled Models

In [24]:
#Linear Models
from sklearn.linear_model import Lasso, ElasticNet


#Kernel Ridge Regression
from sklearn.kernel_ridge import KernelRidge


#Gradient Boosting Machines
from sklearn.ensemble import GradientBoostingRegressor

#Random Forest
from sklearn.ensemble import RandomForestRegressor

#Support Vector Machines
from sklearn import svm

#Others


### Defining Models

Lasso (Elastic went to Lasso)

In [25]:
lasso = make_pipeline(Lasso(alpha =0.0003, random_state=1))
elastic = ElasticNet(alpha= 0.0049, fit_intercept = True, l1_ratio= 0.61)

Kernel Ridge Regression

In [19]:
KRR_deg3 = KernelRidge(alpha=10, kernel='polynomial', degree=3, coef0=4)
KRR_deg2 =KernelRidge(alpha=1, kernel='polynomial', degree=2, coef0=2.5)

Gradient Boost

In [20]:
gbr = GradientBoostingRegressor(max_depth = 2, max_features = 12, min_samples_split = 10, subsample = 0.7,
     random_state=42, learning_rate = 0.01, n_estimators = 4000, verbose = 0)

Random Forest

In [21]:
randomforest1 = RandomForestRegressor(n_estimators=800, max_features=13, random_state=43, oob_score=True,max_depth=7)
randomforest2 = RandomForestRegressor(n_estimators=600, max_features=8, random_state=43, oob_score=True,max_depth=3)

SVM

In [22]:
svm1 = svm.SVR(C=9, epsilon=.009, degree = 1, kernel='poly')
svm2 = svm.SVR(C =11, epsilon = 0.03, gamma = 0.0002, kernel = 'rbf')

##### Ok, let's average models then try a more complex ensemble

### Average Model

Building a class that takes models, fits them, then averages them

In [26]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

Try out a few linear models. General Linear Lasso, Kernel Ridge Regression, and Gradient Boost Regression

In [27]:
averaged_models = AveragingModels(models = (lasso, elastic, KRR_deg3, KRR_deg2, gbr, randomforest1, randomforest2, svm1, svm2))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.1165 (0.0057)



In [28]:
averaged_models.fit(train_x, train_y)

AveragingModels(models=(Pipeline(memory=None,
     steps=[('lasso', Lasso(alpha=0.0003, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))]), ElasticNet(alpha=0.0049, copy_X=True, fit_intercept=T... epsilon=0.03, gamma=0.0002,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)))

In [29]:
predictions = averaged_models.predict(test_x)

In [30]:
prediction = pd.DataFrame({'Id' : (np.arange(len(test_x))+1461),
            'SalePrice': np.exp(predictions)})


In [31]:
prediction.to_csv(path_or_buf="../predictions_avg_data2.csv",index=None)

###### Oh it improves it!!!!!

### Stacking
Well let's move on to the fancy ass ensemble...
Get each model's sale price prediction (on the out of bag fold), and use that as input to another model.

Stacking Class:

In [32]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred.flatten()
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

#### Stack them up, and use a meta model to decide based on the new stack

Using Gboost as meta-model for now in case there are non-linearities

In [35]:
rfmeta = RandomForestRegressor(n_estimators=800)

In [50]:
# First, we ran meta = Random Forests:
# stacked_averaged_models = StackingAveragedModels(base_models = (lasso, elastic, KRR_deg3, KRR_deg2, gbr, randomforest1, randomforest2, svm1, svm2),
#                                                  meta_model = rfmeta)
# Second, we ran meta = lasso
stacked_averaged_models_lasso = StackingAveragedModels(base_models = (lasso, elastic, KRR_deg3, KRR_deg2, gbr, randomforest1, randomforest2, svm1, svm2),
                                                 meta_model = lasso)

score_lasso = rmsle_cv(stacked_averaged_models_lasso)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score_lasso.mean(), score_lasso.std()))


Stacking Averaged models score: 0.1103 (0.0048)


In [ ]:
# for meta_model = lasso we got Stacking Average models score: 0.1103 (0.0048)
# for rfmeta = lasso we got Stacking Average models score: 0.1115 (0.0053)

In [51]:
# stack_rf = stacked_averaged_models
stack_lasso = stacked_averaged_models_lasso

In [52]:
# stack_rf.fit(train_x.values, train_y)
stack_lasso.fit(train_x.values, train_y)

StackingAveragedModels(base_models=(Pipeline(memory=None,
     steps=[('lasso', Lasso(alpha=0.0003, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))]), ElasticNet(alpha=0.0049, copy_X=True, fit_interc... epsilon=0.03, gamma=0.0002,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)),
            meta_model=Pipeline(memory=None,
     steps=[('lasso', Lasso(alpha=0.0003, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))]),
            n_folds=5)

In [53]:
# pred_stack_rf = stack_rf.predict(test_x.values)
pred_stack_lasso = stack_lasso.predict(test_x.values)

In [54]:
# pred_stack_rf
pred_stack_lasso

array([ 11.61397352,  11.98467771,  12.13732433, ...,  12.01012048,
        11.63801922,  12.37002207])

In [55]:
# pred_stack_rf = pd.DataFrame({'Id' : (np.arange(len(test_x))+1461),
#             'SalePrice': np.exp(pred_stack_rf)})
pred_stack_lasso = pd.DataFrame({'Id' : (np.arange(len(test_x))+1461),
            'SalePrice': np.exp(pred_stack_lasso)})


In [56]:
# pred_stack_rf.to_csv(path_or_buf="../Data/predictions_stackrf_data2.csv",index=None)
pred_stack_lasso.to_csv(path_or_buf="../Data/predictions_stacklasso_data2.csv",index=None)

In [57]:
print("Done")

Done
